# What is Pandas
Pandas is an open-source Python library designed for data manipulation and analysis. It makes working with structured data fast, flexible, and intuitive — especially if you're dealing with CSV files, Excel sheets, SQL tables, JSON, or APIs.

It has two core data structures: 
1. `Series:` A 1D labelled array (like a column) 
2. `DataFrame:` A 2D labelled data strcture (like a spreadsheet or SQL table) 

# Key Features:
*   Label-based indexing
*   Column-wise and row-wise operations
*   Support for mixed data types
*   Fast, vectorized operations (built on NumPy)


In [ ]:
import pandas as pd
print(pd.__version__, pd.__file__)

# Creating and Loading a dataframe:

In [ ]:
data = {
    'Gene': ['BRCA0', 'TP53', 'EGFR', 'VEGFA'],
    'Expression': [4.6, 7.2, 3.8, 4.5],
    'Condition': ['Mutated', 'Wild-Type', 'Mutated', 'Wild-Type'],
    'Sample_ID': ['S0', 'S2', 'S3', 'S4']
}

df = pd.DataFrame(data) # R equivalent: data.frame()
print(df)


In [ ]:
# Loading a dataframe

df = pd.read_csv('orders.csv')        # Load from CSV
#df = pd.read_excel('data.xlsx')     # Load from Excel
#df = pd.read_json('data.json')      # Load from JSON

In [ ]:
# head
# tail


In [ ]:
# info
# describe

In [ ]:
# columns
# index

In [ ]:
# slicing rows and columns
# list of columns
# loc and iloc

type(df[["CustomerName", "OrderID"]]) 

print(df.iloc[0:2, 0:3])
print(df.loc[0:1, "OrderID":"Product"])

import numpy as np
#print(df.iloc[np.arange(0,7,2), np.arange(0,5)])


In [ ]:
display(df.loc[0:5, ["CustomerName", "Price", "Product"]].head())
display(df.iloc[:, [1,5,2]].head())

print(df.iloc[0:2, 0:5])
print(df.loc[0:2, "OrderID":"Quantity"])

#print(df.loc[0:1])
#print(df.iloc[0:1])

In [ ]:
# a simple example of Series with custom index
ss = pd.Series(list("abcdefg"), index=[6,7,5,1,2,3,4])
print(ss)

ss = pd.Series(ss.index, index=ss.values)
print(ss)
ss.iloc[1]
ss.loc["b"]


In [ ]:
# find using str
# logical filtering
# multiple conditions

print(df[df["Country"].str.startswith("I")])


In [ ]:
display(df[
    (df["Country"] == "USA") & 
    (df["Category"] == "Electronics")
    ])

In [ ]:
df[~(df["Price"] > 100)]

In [ ]:
display(df[
    (df["Quantity"] > 1) &
    (df["Shipped"] == "Yes")
    ].groupby("Country").sum())

In [ ]:
# aggregations
# groupby

df["Revenue"] = df["Price"] * df["Quantity"]

df.groupby(["Category", "Country"]).agg({"Price":"sum", "Quantity":"max"})

In [ ]:
df.groupby(["Category", "Country"]).agg(
    orders=("OrderID", "nunique"),
    units=("Quantity", "sum"),
    revenue=("Revenue", "sum"),
    avg_price=("Price", "mean")
)

# If you want explicit output column names, use named aggregation with tuple values.
# If you only want to specify aggregations per input column, you can use a dict.

In [ ]:
# pivot table
# pivot long to wide
# melt wide to long

import numpy as np
pivoted = pd.pivot_table(df, index="Country", columns="Category", values="Revenue", aggfunc="sum", fill_value=np.nan)
pivoted

pd.melt(pivoted.reset_index(), id_vars="Country", value_name="Revenue", var_name="Category")

In [ ]:
# handling missing values
# fillna
# dropna , how , axis
print(pivoted.dropna(how="any", axis=0))

In [ ]:
# apply and lambda

def shipped_revenue_share(group):
    total = (group["Quantity"] * group["Price"]).sum()
    shipped = (group.loc[group["Shipped"] == "Yes", "Quantity"]
               * group.loc[group["Shipped"] == "Yes", "Price"]).sum()
    return shipped / total

result = (
    df.groupby("Category")
      .apply(shipped_revenue_share)
      .rename("shipped_revenue_share")
)

result

In [ ]:
# Merge/join/concat

orders = pd.DataFrame({
    "OrderID": [1001, 1002, 1003, 1004],
    "CustomerName": ["John Smith", "Sarah Lee", "John Smith", "Alice Wong"],
    "Product": ["Laptop", "Headphones", "Mouse", "Desk Lamp"],
    "Category": ["Electronics", "Electronics", "Electronics", "Furniture"],
    "Quantity": [1, 2, 1, 3],
    "Price": [1200.0, 150.0, 35.0, 45.0],
    "OrderDate": pd.to_datetime(["2024-06-01", "2024-06-03", "2024-06-11", "2024-06-05"]),
    "Shipped": ["Yes", "No", "Yes", "Yes"],
    "Country": ["USA", "Canada", "USA", "Singapore"],
    "Revenue": [1200.0, 300.0, 35.0, 135.0]
})
orders

In [ ]:
# merge works on top of join, so understanding merge is the key

merged = pd.merge(df, orders, on=["OrderID"], how="left")
merged

pd.merge(df, orders, on=["OrderID", "CustomerName", "Quantity"], how="left").head()

In [ ]:
#Join
# here we need indexes to be aligned
# better to prefer merge over join

df1 = df.set_index("OrderID")
df2 = orders.set_index("OrderID")

df1.join(df2, how="left", lsuffix="_x", rsuffix="_yw").head()

In [ ]:
# concat

pd.concat([df, orders], axis=0) #   R equivalent: rbind
pd.concat([df, orders], axis=1) #   R equivalent: cbind